# Time Series Analysis

In [90]:
# Import necessary libraries
import sys
import os
import logging

# Append the scripts directory to the system path
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Import custom modules
from data_loading import load_data
from data_quality import check_data_quality
from data_cleaning import preprocess_data
from check_data_types import check_data_types
from detect_outliers import detect_outliers

In [91]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [92]:
# Load the data
train_path = '../data/raw/train.csv'
test_path = '../data/raw/test.csv'
store_path = '../data/raw/store.csv'
train, test, store = load_data(train_path, test_path, store_path)

INFO:data_loading:Data loaded successfully


In [94]:
# Train Time series
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [58]:
# Test Time series
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [75]:
# Additional fields in the stores data
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [62]:
# Check data types
train_data_types = check_data_types(train)
test_data_types = check_data_types(test)
print("\nTrain Data Types:")
print(train_data_types)
print("\nTest Data Types:")
print(test_data_types)

INFO:check_data_types:Data types checked
INFO:check_data_types:Data types checked



Train Data Types:
{'Store': dtype('int64'), 'DayOfWeek': dtype('int64'), 'Date': dtype('O'), 'Sales': dtype('int64'), 'Customers': dtype('int64'), 'Open': dtype('int64'), 'Promo': dtype('int64'), 'StateHoliday': dtype('O'), 'SchoolHoliday': dtype('int64')}

Test Data Types:
{'Id': dtype('int64'), 'Store': dtype('int64'), 'DayOfWeek': dtype('int64'), 'Date': dtype('O'), 'Open': dtype('float64'), 'Promo': dtype('int64'), 'StateHoliday': dtype('O'), 'SchoolHoliday': dtype('int64')}


In [63]:
# Detect outliers
train_outliers = detect_outliers(train)
test_outliers = detect_outliers(test)
print("\nTrain Outliers:")
print(train_outliers)
print("\nTest Outliers:")
print(test_outliers)

INFO:detect_outliers:Outliers detected for Store: 0 outliers
INFO:detect_outliers:Outliers detected for DayOfWeek: 0 outliers
INFO:detect_outliers:Outliers detected for Sales: 26694 outliers
INFO:detect_outliers:Outliers detected for Customers: 38095 outliers
INFO:detect_outliers:Outliers detected for Id: 0 outliers
INFO:detect_outliers:Outliers detected for Store: 0 outliers
INFO:detect_outliers:Outliers detected for DayOfWeek: 0 outliers



Train Outliers:
{'Store': {'lower_bound': -557.0, 'upper_bound': 1675.0, 'num_outliers': 0, 'outlier_indices': Index([], dtype='int64')}, 'DayOfWeek': {'lower_bound': -4.0, 'upper_bound': 12.0, 'num_outliers': 0, 'outlier_indices': Index([], dtype='int64')}, 'Sales': {'lower_bound': -2466.5, 'upper_bound': 14049.5, 'num_outliers': 26694, 'outlier_indices': Index([      6,      23,      24,      83,     107,     124,     191,     210,
           250,     258,
       ...
       1015502, 1015541, 1015574, 1015677, 1015735, 1015767, 1015796, 1015821,
       1016093, 1016356],
      dtype='int64', length=26694)}, 'Customers': {'lower_bound': -243.0, 'upper_bound': 1485.0, 'num_outliers': 38095, 'outlier_indices': Index([      3,      24,     124,     210,     250,     258,     261,     319,
           334,     335,
       ...
       1015962, 1015965, 1015993, 1016006, 1016076, 1016093, 1016356, 1016517,
       1016656, 1016827],
      dtype='int64', length=38095)}}

Test Outliers:
{'Id': {

#### Handling Missing values & create new features for further analysis.

In [66]:
# Check initial data quality
train_quality = check_data_quality(train)

logger.info("Initial Train Data Quality Check")
logger.info(train_quality)

INFO:data_quality:Data quality check completed
INFO:__main__:Initial Train Data Quality Check
INFO:__main__:{'head':    Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday  
0              1  
1              1  
2              1  
3              1  
4              1  , 'dtypes': Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object, 'missing_values': Store            0
DayOfWeek      

In [67]:
test_quality = check_data_quality(test)

logger.info("Initial Test Data Quality Check")
logger.info(test_quality)

INFO:data_quality:Data quality check completed
INFO:__main__:Initial Test Data Quality Check
INFO:__main__:{'head':    Id  Store  DayOfWeek        Date  Open  Promo StateHoliday  SchoolHoliday
0   1      1          4  2015-09-17   1.0      1            0              0
1   2      3          4  2015-09-17   1.0      1            0              0
2   3      7          4  2015-09-17   1.0      1            0              0
3   4      8          4  2015-09-17   1.0      1            0              0
4   5      9          4  2015-09-17   1.0      1            0              0, 'dtypes': Id                 int64
Store              int64
DayOfWeek          int64
Date              object
Open             float64
Promo              int64
StateHoliday      object
SchoolHoliday      int64
dtype: object, 'missing_values': Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64}


In [79]:
# Ensure that the index is a DatetimeIndex
if not isinstance(train.index, pd.DatetimeIndex):
    train.index = pd.to_datetime(train.index)

# data extraction
# Extracting month and day from the index
train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekOfYear'] = train.index.isocalendar().week

# Adding a new variable
train['SalePerCustomer'] = train['Sales'] / train['Customers']
train['SalePerCustomer'].describe()


count    844340.000000
mean          9.493619
std           2.197494
min           0.000000
25%           7.895563
50%           9.250000
75%          10.899729
max          64.957854
Name: SalePerCustomer, dtype: float64

#### Closed stores and zero sales stores

In [80]:
# closed stores
train[(train.Open == 0) & (train.Sales == 0)].head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Month,Day,WeekOfYear,SalePerCustomer,Year
1970-01-01 00:00:00.000000291,292,5,2015-07-31,0,0,0,1,0,1,1,1,1,NaN,1970
1970-01-01 00:00:00.000000875,876,5,2015-07-31,0,0,0,1,0,1,1,1,1,NaN,1970
1970-01-01 00:00:00.000001406,292,4,2015-07-30,0,0,0,1,0,1,1,1,1,NaN,1970
1970-01-01 00:00:00.000001990,876,4,2015-07-30,0,0,0,1,0,1,1,1,1,NaN,1970
1970-01-01 00:00:00.000002521,292,3,2015-07-29,0,0,0,1,0,1,1,1,1,NaN,1970


In [81]:
# opened stores with zero sales
zero_sales = train[(train.Open != 0) & (train.Sales == 0)]
print("In total: ", zero_sales.shape)
zero_sales.head(5)

In total:  (54, 14)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Month,Day,WeekOfYear,SalePerCustomer,Year
1970-01-01 00:00:00.000086825,971,5,2015-05-15,0,0,1,0,0,1,1,1,1,NaN,1970
1970-01-01 00:00:00.000142278,674,4,2015-03-26,0,0,1,0,0,0,1,1,1,NaN,1970
1970-01-01 00:00:00.000196938,699,4,2015-02-05,0,0,1,1,0,0,1,1,1,NaN,1970
1970-01-01 00:00:00.000322053,708,3,2014-10-01,0,0,1,1,0,0,1,1,1,NaN,1970
1970-01-01 00:00:00.000330176,357,1,2014-09-22,0,0,1,0,0,0,1,1,1,NaN,1970


In [83]:
print("Closed stores and days which didn't have any sales won't be counted into the forecasts.")
train = train[(train["Open"] != 0) & (train['Sales'] != 0)]

print("In total: ", train.shape)

Closed stores and days which didn't have any sales won't be counted into the forecasts.
In total:  (844338, 14)


In [95]:
# Get additional information about the stores from store
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [89]:
# Check initial data quality
store_quality = check_data_quality(store)
logger.info("Initial Train Data Quality Check")
logger.info(train_quality)
store.isnull().sum()

INFO:data_quality:Data quality check completed
INFO:__main__:Initial Train Data Quality Check
INFO:__main__:{'head':    Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday  
0              1  
1              1  
2              1  
3              1  
4              1  , 'dtypes': Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object, 'missing_values': Store            0
DayOfWeek      

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            0
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [96]:
# fill NaN with a median value (skewed distribuion)
store['CompetitionDistance'].fillna(store['CompetitionDistance'].median(), inplace = True)

In [98]:
# no promo = no information about the promo?
_ = store[pd.isnull(store.Promo2SinceWeek)]
_[_.Promo2 != 0].shape

(0, 10)

In [99]:
# replace NA's by 0
store.fillna(0, inplace = True)

#### Joining train set with an additional store information.

In [100]:
train_store = pd.merge(train, store, how = 'inner', on = 'Store')
print("In total: ", train_store.shape)
train_store.head()

In total:  (1017209, 18)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0


#### Store types

In [105]:
train_store.groupby('StoreType')['Sales'].describe()

,count,mean,std,min,25%,50%,75%,max
StoreType,,,,,,,,
a,551627.0,5738.179710,3964.028134,0.0,3508.0,5618.0,7860.00,41551.0
b,15830.0,10058.837334,5280.525506,0.0,6227.0,9025.5,13082.75,38722.0
c,136840.0,5723.629246,3721.700886,0.0,3789.0,5766.0,7849.00,31448.0
d,312912.0,5641.819243,3473.393781,0.0,3986.0,5826.0,7691.00,38037.0


In [107]:
# Grouping by 'StoreType' and summing 'Customers' and 'Sales'
grouped_data = train_store.groupby('StoreType')[['Customers', 'Sales']].sum()
grouped_data.head()

,Customers,Sales
StoreType,,
a,363541434,3165334859
b,31465621,159231395
c,92129705,783221426
d,156904995,1765392943


In [110]:
train_store.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0


##### There are several things here:
- In case of no promotion, both `Promo` and `Promo2` are equal to 0, `Sales` tend to peak on Sunday (!). Though we should note that `StoreType` C doesn't work on Sundays. So it is mainly data from `StoreType` A, B and D.
- On the contrary, stores that run the promotion tend to make most of the `Sales` on Monday. This fact could be a good indicator for Rossmann marketing campaigns. The same trend follow the stores which have both promotion at the same time (`Promo` and `Promo2` are equal to 1).
- `Promo2` alone doesn't seem to be correlated to any significant change in the `Sales` amount. This can be also prooved by the blue pale area on the heatmap above.

---

### Conclusion of EDA

- The most selling and crowded `StoreType` is A.


- The best "Sale per Customer" `StoreType` D indicates to the higher Buyer Cart. To benefit from this fact, Rossmann can consider proposing bigger variety of its products.


- Low `SalePerCustomer` amount for `StoreType` B indicates to the possible fact that people shop there essentially for "small" things. Eventhough this `StoreType` generated the least amount of sales and customers over the whole period, it shows a great potential.


- Customers tends to buy more on Modays when there's one promotion (`Promo`) and on Sundays when there's no promotion at all (both `Promo` and `Promo1` are equal to 0).


- Promo2 alone doesn't seem to be correlated to any significant change in the `Sales` amount.

<br>
## Time-Series Analysis per Store Type

What makes a time series different from a regular regression problem? 

- It is time dependent. The basic assumption of a linear regression that the observations are independent doesn’t hold in this case.
- Along with an increasing or decreasing trend, most time series have some form of seasonality trends, i.e. variations specific to a particular time frame. For example, for Christmas holidays, which we will see in this dataset.

We build a time series analysis on store types instead of individual stores. The main advantage of this approach is its simplicity of presentation and overall account for different trends and seasonalities in the dataset. 